In [1]:
import pickle

import keras.models as k_mod
import keras.layers as k_lay
import keras.optimizers as k_opt
import keras.utils as k_ut
import numpy as np
import pandas as pd
import sklearn.metrics as sk_mt
import sklearn.model_selection as sk_ms
import sklearn.preprocessing as sk_pp
import sklearn.utils as sk_ut

Using TensorFlow backend.


In [2]:
# https://drive.google.com/open?id=1bDYJj9HkiRUOpzH2-D_ZQbji5LUeu_fJ
with open('tweets_net.pkl', 'rb') as f:
    ds = pickle.load(f)

In [3]:
x_vec = [[wx[1] for wx in sx]for sx in ds['x']]
data = list(zip(x_vec, ds['y']))
data = [ex for ex in data if len(ex[0]) > 0]
data = [(np.average(ex[0], axis=0), ex[1]) for ex in data]
# data = np.array(data)
x, y = zip(*data)

In [4]:
x = np.array(x)
y = np.array(y)
lb = sk_pp.LabelBinarizer()
lb.fit(y)
x_train, x_test, y_train, y_test = sk_ms.train_test_split(x, lb.transform(y), shuffle=True)

In [5]:
input_dim = len(x[0])
output_dim = len(set(y))

In [6]:
model = k_mod.Sequential()
model.add(k_lay.Dense(64, activation='relu', input_dim=input_dim))
model.add(k_lay.Dropout(0.5))
model.add(k_lay.Dense(32, activation='relu'))
model.add(k_lay.Dropout(0.5))
model.add(k_lay.Dense(16, activation='relu'))
model.add(k_lay.Dropout(0.5))
model.add(k_lay.Dense(output_dim, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=k_opt.Adam(0.001), metrics=['accuracy'])

In [7]:
model.fit(x_train, y_train, epochs=50, batch_size=128)

Epoch 1/50
14133/14133 [==============================] - 1s 76us/step - loss: 1.3978 - acc: 0.2789
Epoch 2/50
14133/14133 [==============================] - 1s 43us/step - loss: 1.3425 - acc: 0.3484
Epoch 3/50
14133/14133 [==============================] - 1s 47us/step - loss: 1.2839 - acc: 0.3853
Epoch 4/50
14133/14133 [==============================] - 1s 38us/step - loss: 1.2564 - acc: 0.4008
Epoch 5/50
14133/14133 [==============================] - 0s 32us/step - loss: 1.2368 - acc: 0.4105
Epoch 6/50
14133/14133 [==============================] - 0s 24us/step - loss: 1.2241 - acc: 0.4258
Epoch 7/50
14133/14133 [==============================] - 0s 35us/step - loss: 1.2144 - acc: 0.4339
Epoch 8/50
14133/14133 [==============================] - 0s 30us/step - loss: 1.2054 - acc: 0.4383
Epoch 9/50
14133/14133 [==============================] - 0s 31us/step - loss: 1.1952 - acc: 0.4495
Epoch 10/50
14133/14133 [==============================] - 0s 31us/step - loss: 1.1875 - acc: 0.4625

In [8]:
pred = model.predict_classes(x_test)
y_pred = lb.inverse_transform(k_ut.to_categorical(pred))

In [9]:
print(sk_mt.classification_report(lb.inverse_transform(y_test), y_pred))

              precision    recall  f1-score   support

      actors       0.46      0.36      0.40      1216
     athlets       0.41      0.60      0.49      1177
    musicans       0.54      0.37      0.44      1118
  politicans       0.68      0.73      0.70      1201

   micro avg       0.52      0.52      0.52      4712
   macro avg       0.52      0.51      0.51      4712
weighted avg       0.52      0.52      0.51      4712

